##### Output Parser là dạng format cho hậu xử lý gồm nhiều loại:
##### Json, String Text, ...
##### Dùng Pydantic library để validation output format



In [32]:
from configparser import ConfigParser
import os
config = ConfigParser()
config.read("./config.ini")
os.environ["OPENAI_API_KEY"]=config["KEY"]["openai_key"]

In [22]:
from pydantic import BaseModel, Field

## Using Output Json

In [23]:
from langchain_core.output_parsers import JsonOutputParser

In [24]:
class Joke(BaseModel):
        setup: str = Field(description="question to set up a joke")
        punchline: str = Field(description="answer to resolve the joke")

parser = JsonOutputParser(pydantic_object=Joke)

In [25]:
parser

JsonOutputParser(pydantic_object=<class '__main__.Joke'>)

In [30]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"setup": {"description": "question to set up a joke", "title": "Setup", "type": "string"}, "punchline": {"description": "answer to resolve the joke", "title": "Punchline", "type": "string"}}, "required": ["setup", "punchline"]}
```


In [27]:
from langchain_core.prompts import PromptTemplate
prompt = PromptTemplate(
        template="Answer the user query \n{format_instructions}\n{query}\n",
        input_variables=["query"],
        partial_variables={"format_instructions":parser.get_format_instructions()}
)


In [29]:
print(prompt)

input_variables=['query'] input_types={} partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"setup": {"description": "question to set up a joke", "title": "Setup", "type": "string"}, "punchline": {"description": "answer to resolve the joke", "title": "Punchline", "type": "string"}}, "required": ["setup", "punchline"]}\n```'} template='Answer the user query \n{format_instructions}\n{query}\n'


In [31]:
from langchain_openai import ChatOpenAI

In [33]:
llm = ChatOpenAI(
        api_key=os.environ["OPENAI_API_KEY"],
        temperature=0.2,
        max_tokens=100
)

In [35]:
chain = prompt | llm

output = chain.invoke({"query":"tell me a joke"})
parser_output = parser.invoke(output)
print(parser_output)

{'setup': "Why couldn't the bicycle stand up by itself?", 'punchline': 'Because it was two tired!'}


In [36]:
type(parser_output)

dict